In [3]:
import torch
import sys
sys.path.append('..')

checkpoint = torch.load('/n/groups/patel/tom/PhE-T/ckpts/PhE-T/v0/best-epoch=3-step=3842.ckpt', map_location=torch.device('cpu'))
tokenizer = checkpoint['hyper_parameters']['tokenizer']

/tmp/ipykernel_21782/3161387479.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/n/groups/patel/tom/PhE-T/ckpts/PhE-T/v0/best-epoch=3-step=3842.

In [4]:
tokenizer.p_name_map

{0: 'Special tokens',
 1: 'BMI',
 2: 'HDL cholesterol',
 3: 'LDL cholesterol',
 4: 'Total cholesterol',
 5: 'Triglycerides',
 6: 'Diastolic blood pressure',
 7: 'Age',
 8: 'Sex',
 9: 'Ever smoked',
 10: 'Snoring',
 11: 'Insomnia',
 12: 'Daytime napping',
 13: 'Chronotype',
 14: 'Sleep duration',
 15: 'Asthma',
 16: 'Cataract',
 17: 'Diabetes',
 18: 'GERD',
 19: 'Hay-fever & Eczema',
 20: 'Major depression',
 21: 'Myocardial infarction',
 22: 'Osteoarthritis',
 23: 'Pneumonia',
 24: 'Stroke'}

In [87]:
num_features = [
    'BMI',
    'HDL cholesterol',
    'LDL cholesterol',
    'Total cholesterol',
    'Triglycerides',
    'Diastolic blood pressure'
]
cat_features = [
    'Age',
    'Sex',
    'Ever smoked',
    'Snoring',
    'Insomnia',
    'Daytime napping',
    'Chronotype',
    'Sleep duration',
]

tab_path = '../data/val.csv'
spiro_path = '../data/val_spiro.pkl'
features = num_features + cat_features
dataset = TabSpiroDataset(tab_path, spiro_path, features, tokenizer)
data_loader = DataLoader(
    dataset,
    batch_size = 16,
    shuffle = False,
    num_workers = 1,
    pin_memory = False
)

Tokenize data: 100%|██████████| 7453/7453 [00:02<00:00, 3411.89it/s]


In [88]:
for x in data_loader:
    batch = x
    break

In [89]:
batch['label']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [77]:
import torch.nn as nn
from torch.nn import functional as F
from src.phet import PhET
from src.resnet import ResNet18D1D



        

In [78]:
ckpt_phet = '/n/groups/patel/tom/PhE-T/ckpts/PhE-T/v0/best-epoch=3-step=3842.ckpt'
ckpt_resnet = '/n/groups/patel/tom/PhE-T/ckpts/AsthmaResNet/v2/best-epoch=7-step=3650.ckpt'
model = AsthmaPhET(ckpt_phet, ckpt_resnet, freeze_phet=True, freeze_resnet=True)

/tmp/ipykernel_23984/3533138689.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_phet, map_location=torch.device('cpu'))
/n/groups/patel/tom

In [79]:
model(value_ids=batch['value_ids'], phenotype_ids=batch['phenotype_ids'], flow_volume=batch['flow_volume'].unsqueeze(1), mask=batch['mask'])

{'loss': tensor(12.0613, grad_fn=<NllLossBackward0>),
 'logits': tensor([[-0.1596, -0.5027, -0.9349,  ...,  2.7880,  0.8484,  0.9928],
         [ 0.2341, -0.2763, -0.5157,  ...,  1.9422,  1.0593,  0.3096],
         [ 0.5305,  0.2170,  0.5540,  ...,  2.6339,  1.8237,  0.2373],
         ...,
         [ 0.2653, -1.5970, -1.7812,  ...,  3.0984,  1.4356,  0.7236],
         [ 0.0892, -1.5520, -1.8103,  ...,  4.1489,  2.3216,  2.1648],
         [ 0.0329, -0.6312, -0.3315,  ...,  3.8841,  1.7249,  1.2385]],
        grad_fn=<IndexBackward0>),
 'score': tensor([0.2772, 0.3211, 0.1903, 0.1311, 0.1617, 0.1482, 0.2380, 0.1705, 0.1924,
         0.1562, 0.1076, 0.2286, 0.1310, 0.2151, 0.1680, 0.1553],
        grad_fn=<DivBackward0>)}